In [1]:
# first create a table in my kickstarter database
# test that I can add the raw html
# test that I can retrieve it and then process with beautiful soup
# use the same primary key as my other
# columns - project and creator pages

In [42]:
import requests
from bs4 import BeautifulSoup
import pprint
import pymysql as mdb
import re
import pandas as pd
import numpy as np
from unidecode import unidecode

In [139]:
#soup = BeautifulSoup(p.text)
#print soup.prettify()
#print soup.get_text() #looks great!

In [140]:
con = mdb.connect('localhost','root','hobbes','kickstarter');

In [141]:
with con:
    
    cur = con.cursor()
    cur.execute("SHOW DATABASES")
    
    rows = cur.fetchall()
    for row in rows:
        print row

('information_schema',)
('kickscrape',)
('kickstarter',)
('mysql',)
('performance_schema',)
('test',)
('testdb',)
('world_innodb',)


In [142]:
# Create and describe my new table
# with con:
#     cur = con.cursor()
#     cur.execute("DROP TABLE IF EXISTS Scraped_pages")
#     cur.execute("CREATE TABLE Scraped_pages(Id INT PRIMARY KEY AUTO_INCREMENT,\
#                  Project_html LONGTEXT,\
#                  Creator_html LONGTEXT)")
    
#     cur.execute("DESCRIBE Scraped_pages")
#     rows = cur.fetchall()
    
#     for row in rows:
#         print row[0:3]
#     cur.close()

('Id', 'int(11)', 'NO')
('Project_html', 'longtext', 'YES')
('Creator_html', 'longtext', 'YES')


In [143]:
#start by scraping all techonology kickstarters

In [144]:
with con:
    cur = con.cursor()
    cur.execute("SELECT Id, Creator_url, URL FROM Project_info \
                WHERE Category_slug LIKE 'technology%'")
    tech_id_urls = cur.fetchall()
    cur.close()

In [146]:
len(tech_id_urls)

9351

In [152]:
print tech_id_urls

((14732, 'https://www.kickstarter.com/profile/2005894276', 'https://www.kickstarter.com/projects/2005894276/beenmissing-we-are-looking-for-you?ref=category'), (14733, 'https://www.kickstarter.com/profile/461687407', 'https://www.kickstarter.com/projects/461687407/kickstarter-open-source-death-star?ref=category'), (14734, 'https://www.kickstarter.com/profile/345654486', 'https://www.kickstarter.com/projects/345654486/national-biker-network?ref=category'), (14735, 'https://www.kickstarter.com/profile/customapps', 'https://www.kickstarter.com/projects/customapps/ebola-news-updates-and-maps-app-ios-android-and-wi?ref=category'), (14736, 'https://www.kickstarter.com/profile/1935113228', 'https://www.kickstarter.com/projects/1935113228/t-rex-multi-flex-diver?ref=category'), (14737, 'https://www.kickstarter.com/profile/395987959', 'https://www.kickstarter.com/projects/395987959/occupant-protection-system?ref=category'), (14738, 'https://www.kickstarter.com/profile/1168069859', 'https://www.ki

In [153]:
with con:
    cur = con.cursor()
    
    for i,id_urls in enumerate(tech_id_urls[7612:]): 
        if not i%50:
            print i
            
        primary_key, creator_url, project_url = id_urls
        project_url = project_url.replace('?ref=category','/description')
    
        project_request = requests.get(project_url)
        creator_request = requests.get(creator_url)
        project_raw = unidecode(project_request.text)
        creator_raw = unidecode(creator_request.text)
        
        cur.execute("INSERT INTO Scraped_pages () VALUES(%s,%s,%s)",\
            [primary_key,project_raw,creator_raw])
        
        con.commit()
    cur.close()

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700


In [150]:
# where did I get kicked off?
with con:
    cur = con.cursor()
    
    
    cur.execute("SELECT COUNT(Id) FROM Scraped_pages")
    rows = cur.fetchall()
    print rows
    cur.close()

((7611,),)
